In [ ]:
import glob
from pathlib import Path

import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

plt.style.use("seaborn-v0_8-paper")
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
# Load data
df = pd.read_csv("result/convergence_results.csv")
h = df["h"].values
err_mb = df["error_mb"].values
err_mf = df["error_mf"].values

# Setup plot
plt.figure(figsize=(10, 7))

# Plot Solvers
plt.loglog(h, err_mb, "o-", linewidth=2, label="Matrix-Based", markersize=8)
plt.loglog(h, err_mf, "s--", linewidth=2, label="Matrix-Free", markersize=8)

# Add Reference Lines (Theoretical Order)
# For FE Degree 2:
# L2 Error should be O(h^(p+1)) = O(h^3)
# H1 Error should be O(h^p)     = O(h^2)

# We create a reference line aligned with the last data point
ref_h = h
# O(h^3) Reference
ref_error_3 = ref_h**3
plt.loglog(ref_h, ref_error_3, "k:", linewidth=1.5, label="Reference O($h^3$)")

# O(h^2) Reference (optional, usually for H1 norm)
ref_error_2 = ref_h**2
plt.loglog(
    ref_h, ref_error_2, "k-.", linewidth=1.0, alpha=0.5, label="Reference O($h^2$)"
)

# Formatting
plt.title("Convergence Study: Advection-Diffusion-Reaction", fontsize=16)
plt.xlabel("Grid Size ($h$)", fontsize=14)
plt.ylabel("$L_2$ Error Norm", fontsize=14)
plt.grid(True, which="both", ls="-", alpha=0.4)
plt.legend(fontsize=12)

# Invert X axis so smaller h is on the right?
# Standard convention is smaller h on left, but loglog handles scale automatically.
# Usually we want h decreasing from right to left or just standard increasing x.
# Matplotlib default is increasing x (small h on left).

plt.tight_layout()
plt.savefig("./report/figures/convergence_plot.pdf", dpi=300)
plt.show()

# Print Convergence Rates
print("\nCalculated Convergence Rates (Matrix-Free):")
for i in range(1, len(h)):
    rate = np.log(err_mf[i - 1] / err_mf[i]) / np.log(h[i - 1] / h[i])
    print(f"Refinement {i + 2} -> {i + 3}: Rate = {rate:.4f}")

In [ ]:
# Read all CSV files
csv_files = glob.glob("result/complexity_*.csv")
# Combine all results
all_results = []
for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    config_name = Path(csv_file).stem.replace("complexity_", "")
    df["config"] = config_name
    all_results.append(df)
results_df = pd.concat(all_results, ignore_index=True)
results_df.head()

In [ ]:
from itertools import cycle

import matplotlib.pyplot as plt

palette = sns.color_palette("colorblind")
colors = cycle(palette)

linestyles = cycle(["-", "--", "-.", ":"])
markers = cycle(["o", "s", "D", "^", "v", "P"])

fig = plt.figure(figsize=(10, 7))

for config in sorted(results_df["config"].unique()):
    config_data = results_df[results_df["config"] == config]
    mb = config_data[config_data["solver_type"] == "matrix_based"].sort_values("n_dofs")

    if not mb.empty:
        plt.plot(
            mb["n_dofs"],
            mb["total_time"],
            label=config,
            color=next(colors),
            linestyle=next(linestyles),
            marker=next(markers),
            linewidth=2,
            markersize=5,
        )

plt.xlabel("Degrees of Freedom")
plt.ylabel("Total Time (seconds)")
plt.title("Matrix-Based Solver: Time Complexity")

plt.xscale("log")
plt.yscale("log")

plt.legend(frameon=False, fontsize=9)
plt.grid(True, which="both", linestyle=":", alpha=0.3)
plt.savefig("./report/figures/complexity_matrix_based.pdf", dpi=300)
plt.tight_layout()
plt.show()

In [ ]:
from itertools import cycle

import matplotlib.pyplot as plt

palette = sns.color_palette("colorblind")
colors = cycle(palette)

linestyles = cycle(["-", "--", "-.", ":"])
markers = cycle(["o", "s", "D", "^", "v", "P"])

fig = plt.figure(figsize=(10, 7))

for config in sorted(results_df["config"].unique()):
    config_data = results_df[results_df["config"] == config]
    mb = config_data[config_data["solver_type"] == "matrix_free"].sort_values("n_dofs")

    if not mb.empty:
        plt.plot(
            mb["n_dofs"],
            mb["total_time"],
            label=config,
            color=next(colors),
            linestyle=next(linestyles),
            marker=next(markers),
            linewidth=2,
            markersize=5,
        )

plt.xlabel("Degrees of Freedom")
plt.ylabel("Total Time (seconds)")
plt.title("Matrix-Free Solver: Time Complexity")

plt.xscale("log")
plt.yscale("log")

plt.legend(frameon=False, fontsize=9)
plt.grid(True, which="both", linestyle=":", alpha=0.3)
plt.savefig("./report/figures/complexity_matrix_free.pdf", dpi=300)
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# 1. Load Data
files = [
    "result/scaling_results_2d_1mpi_1thr.csv",
    "result/scaling_results_2d_2mpi_1thr.csv",
    "result/scaling_results_2d_4mpi_1thr.csv",
    "result/scaling_results_2d_8mpi_1thr.csv",
]

df_list = []
for f in files:
    try:
        df = pd.read_csv(f)
        df_list.append(df)
    except Exception as e:
        print(f"Error reading {f}: {e}")

if not df_list:
    raise ValueError("No data loaded.")

data = pd.concat(df_list, ignore_index=True)

# Sort by solver_type and total_cores
data.sort_values(by=["solver_type", "total_cores"], inplace=True)

# 2. Calculate Speedup and Efficiency
# Baseline is 1 core (n_mpi=1)
baselines = data[data["total_cores"] == 1].set_index("solver_type")["total_time_avg"]


def calculate_metrics(row):
    base_time = baselines.get(row["solver_type"])
    if base_time is not None and not pd.isna(base_time):
        speedup = base_time / row["total_time_avg"]
        efficiency = (speedup / row["total_cores"]) * 100
        return pd.Series(
            [speedup, efficiency], index=["calc_speedup", "calc_efficiency"]
        )
    return pd.Series([np.nan, np.nan], index=["calc_speedup", "calc_efficiency"])


data[["calc_speedup", "calc_efficiency"]] = data.apply(calculate_metrics, axis=1)

# 3. Print Summary
print("--- Data Summary ---")
cols = [
    "solver_type",
    "total_cores",
    "total_time_avg",
    "calc_speedup",
    "calc_efficiency",
    "memory_mb",
]
print(data[cols].to_string(index=False))

# 4. Plotting
sns.set(style="whitegrid", context="talk")
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# --- Plot 1: Total Execution Time ---
sns.lineplot(
    ax=axes[0, 0],
    data=data,
    x="total_cores",
    y="total_time_avg",
    hue="solver_type",
    marker="o",
    linewidth=2.5,
)
axes[0, 0].set_title("Total Execution Time vs Cores")
axes[0, 0].set_ylabel("Time (s)")
axes[0, 0].set_xlabel("Number of Cores")
axes[0, 0].set_xticks(sorted(data["total_cores"].unique()))

# --- Plot 2: Speedup ---
sns.lineplot(
    ax=axes[0, 1],
    data=data,
    x="total_cores",
    y="calc_speedup",
    hue="solver_type",
    marker="o",
    linewidth=2.5,
)
# Ideal line
max_cores = data["total_cores"].max()
axes[0, 1].plot([1, max_cores], [1, max_cores], "k--", label="Ideal", alpha=0.5)
axes[0, 1].set_title("Strong Scaling Speedup")
axes[0, 1].set_ylabel("Speedup")
axes[0, 1].set_xlabel("Number of Cores")
axes[0, 1].set_xticks(sorted(data["total_cores"].unique()))
axes[0, 1].legend()

# --- Plot 3: Efficiency ---
sns.lineplot(
    ax=axes[1, 0],
    data=data,
    x="total_cores",
    y="calc_efficiency",
    hue="solver_type",
    marker="o",
    linewidth=2.5,
)
axes[1, 0].set_title("Parallel Efficiency")
axes[1, 0].set_ylabel("Efficiency (%)")
axes[1, 0].set_xlabel("Number of Cores")
axes[1, 0].set_xticks(sorted(data["total_cores"].unique()))
axes[1, 0].set_ylim(0, 120)
axes[1, 0].axhline(100, color="gray", linestyle="--", alpha=0.5)

# --- Plot 4: Memory Usage ---
sns.barplot(ax=axes[1, 1], data=data, x="total_cores", y="memory_mb", hue="solver_type")
axes[1, 1].set_title("Memory Usage (MB)")
axes[1, 1].set_ylabel("Memory (MB)")
axes[1, 1].set_xlabel("Number of Cores")

plt.tight_layout()
plt.savefig("scaling_analysis_summary.pdf")
plt.show()

# --- Plot 5: Time Breakdown ---
melted_time = data.melt(
    id_vars=["solver_type", "total_cores"],
    value_vars=["setup_time_avg", "assembly_time_avg", "solve_time_avg"],
    var_name="Component",
    value_name="Time",
)

g = sns.catplot(
    data=melted_time,
    x="total_cores",
    y="Time",
    hue="Component",
    col="solver_type",
    kind="bar",
    height=5,
    aspect=1.2,
    palette="muted",
)
g.fig.suptitle("Time Breakdown: Setup vs Assembly vs Solve", y=1.05)
plt.savefig("scaling_breakdown.pdf")
plt.show()